# Aemet OpenData
Aemet makes available to the interested public meteorological data from some of its stations through the OpenData service (https://opendata.aemet.es/centrodedescargas/inicio).

In this notebook the aemet_open_data module is used to facilitate the download of meteorological data. Depending on the number of stations requested or the time period of the data, it may be necessary to make several requests to the Aemet server. The result of each request is saved in a CSV file. Once the process is finished, the module includes an utility to join the downloaded files with data of the same type. 

To start using the service you have to create a free key in (https://opendata.aemet.es/centrodedescargas/inicio) and save it in the apikey.txt file. 

In [2]:
from datetime import date

from aemet_open_data import AemetOpenData

## User-supplied parameters.
The methods used to download data require some values to be supplied by the user. Not all methods require the same parameters.

* __Parameters__
* d1 (date): Data will be requested between date d1 and date d2; d1 is the start date
* d2 (date): Data will be requested between date d1 and date d2; d2 is the end date
* dir_path (str): directory path where files will be saved
* fetch (str): For a set of variables, controls whether data, metadata or both will be downloaded; a value in ('data', 'metadata' or 'both')
* stations (list of str): List of meteorological stations from which data is requested
* time_step (str): Temporal resolution of meteorological data to be downloaded: 'day' or 'month'
* use_files (bool), optional, default True: If True data saved in previously downloaded files not be requested again 
* verbose (bool), optional, default True: Controls the number of messages in the screen. If True all messages will be displayed

The notebook is organized in pairs of cells:
* In the first one the parameters are given values.
* In the second cell the corresponding method is executed

## Meteorological stations in OpenData

In [2]:
# parameters
dir_path = './download/meteo_stations'  
fetch = 'both'  
use_files = True  
verbose = True  

In [3]:
aod = AemetOpenData()
aod.meteo_stations(dir_path, fetch, use_files, verbose)

estaciones_open_data_data.csv has been previously downloaded
estaciones_open_data_metadata.csv has been previously downloaded


## Daily meteorological data from all stations

In [3]:
# parameters
d1 = date(2023, 1, 1)
d2 = date(2023, 8, 1)
dir_path = './download/data_all_stations'
fetch ='both'  
verbose = True  
use_files = True  

In [4]:
aod = AemetOpenData()
downloaded_files = aod.meteo_data_all_stations(d1, d2, dir_path, fetch, verbose, use_files)
print('Downloaded files: ', len(downloaded_files))

stations_20230101T000000UTC_20230201T235959UTC_data.csv: 200, OK 
stations_20230101T000000UTC_20230201T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230202T000000UTC_20230305T235959UTC_data.csv: 200, OK 
stations_20230202T000000UTC_20230305T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230306T000000UTC_20230406T235959UTC_data.csv: 200, OK 
stations_20230306T000000UTC_20230406T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230407T000000UTC_20230508T235959UTC_data.csv: 200, OK 
stations_20230407T000000UTC_20230508T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230509T000000UTC_20230609T235959UTC_data.csv: 200, OK 
stations_20230509T000000UTC_20230609T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230610T000000UTC_20230711T235959UTC_data.csv: 200, OK 
stations_20230610T000000UTC_20230711T235959UTC_metadata.csv: 200, OK Climatologías diarias
stations_20230712T000000UTC_20230801T235959UTC_data.csv: 2

## Daily or monthly meteorological data from selected stations

In [10]:
#parameters
time_step = 'month'
d1 = date(2019, 1, 1)
d2 = date(2019, 3, 31)
stations = ['7178I', '7031', '7031X']
dir_path = './download/data_selected_stations'
fetch = 'both'
verbose = True
use_files = True

In [11]:
aod = AemetOpenData()
downloaded_files = aod.meteo_data_by_station(time_step, d1, d2, stations, dir_path, fetch, verbose, use_files)
print('Downloaded files: ', len(downloaded_files))

7178I_2019_2019_data.csv: 200, OK 
7178I_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
7031_2019_2019_data.csv: 200, OK 
7031_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
7031X_2019_2019_data.csv: 200, OK 
7031X_2019_2019_metadata.csv: 200, OK Climatologías mensuales anuales
Downloaded files:  6


## Export downloaded meteorological data files to a Sqlite3 database 
If the data request to Aemet generated many files, the AOD_2db module can be used to concatenate them into one.
The module saves the data in a Sqlite3 database; once the table is created it can be exported to a csv file.<br>
A specific database is created according to how the data request was made to the Aemet server: 
* Daily weather data from all stations.
* Daily weather data from selected stations.
* Monthly weather data from selected stations.
Each database has only one table.

In [1]:
from aod_2db import AOD_2db

### Create a database

In [1]:
# parameters        
dpath = './download'  # directory with csv files
"""
type of request to Aemet: 
* 'stations_day'. Daily meteorological data from all stations
* 'station1_day'. Daily meteorological data from selected stations
* 'station1_month'. Monthly meteorological data from selected stations
"""
ftype = 'stations_day'  

In [3]:
a2db = AOD_2db(dpath, ftype)
if not a2db.to_db():
    print('Attemp failed')


C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db
already exists, overwrite (y/n)?:  y


0 stations_20230101T000000UTC_20230201T235959UTC_data.csv
1 stations_20230202T000000UTC_20230305T235959UTC_data.csv
2 stations_20230306T000000UTC_20230406T235959UTC_data.csv
3 stations_20230407T000000UTC_20230508T235959UTC_data.csv
4 stations_20230509T000000UTC_20230609T235959UTC_data.csv
5 stations_20230610T000000UTC_20230711T235959UTC_data.csv
6 stations_20230712T000000UTC_20230801T235959UTC_data.csv

Previous downloaded files has been imported to the sqlite db
C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.db


### Export database to a csv file


In [2]:
# parameters
dpath = './download'  # directory path where the database is stored
ftype = 'stations_day'
a2db = AOD_2db(dpath, ftype)

In [3]:
a2db.to_csv()


C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.csv exists, continue: ?  y


Data dumped to
C:\Users\solis\Documents\DEV\python3\meteoro\aemet_open_data\download\metd_all_stations.csv


True